In [1]:
import pandas as pd
import time
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd

In [2]:
query = 'SELECT rede,sigla_uf,  id_municipio, sum(alunos_soma) as alunos_soma, sum(professores_soma) as professores_soma FROM `repositoriodedadosgpsp.Datalake.INEP_media_aluno_docente` where ano=2020 group by rede,sigla_uf,  id_municipio'

In [3]:
client = bigquery.Client()

In [4]:

df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████| 14578/14578 [00:01<00:00, 10780.49rows/s]


In [5]:
df

,rede,sigla_uf,id_municipio,alunos_soma,professores_soma
0,federal,DF,5300108,48376,2300
1,federal,AC,1200609,3103,158
2,federal,RR,1400100,10554,533
3,federal,AC,1200708,3093,142
4,federal,RR,1400209,2999,172
...,...,...,...,...,...
14573,municipal,GO,5214804,58,4
14574,municipal,MG,3118700,15,2
14575,municipal,MG,3143153,179,10
14576,municipal,SP,3539905,172,12


In [6]:
sum(df['professores_soma'])

8746720

In [7]:
## antigo

sum(df['professores_soma'])

8746720

In [38]:
## Subindo para o GBQ

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14987 entries, 0 to 14986
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   rede              14987 non-null  object
 1   sigla_uf          14987 non-null  object
 2   id_municipio      14987 non-null  object
 3   alunos_soma       14987 non-null  Int64 
 4   professores_soma  14987 non-null  Int64 
dtypes: Int64(2), object(3)
memory usage: 614.8+ KB


In [15]:


schema=[bigquery.SchemaField('rede','STRING',description='Rede'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('id_municipio','STRING',description='ID do município no IBGE'),
 bigquery.SchemaField('alunos_soma','INTEGER',description='Soma de aluno'),
 bigquery.SchemaField('professores_soma','INTEGER',description='Soma de aluno')
 ]

In [16]:
dataset_ref = client.dataset('perfil_remuneracao')
table_ref = dataset_ref.table('INEP_media_aluno_docente_rede')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=fb32e835-1156-4d8a-b5e5-e00a193806ea>